# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Imports and Libraries 
import pandas as pd
import numpy as np

import re
import os
import glob
import json
import csv

import cassandra

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

In [26]:
# Utils and Checks

# Uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))

# Uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list[0])

In [5]:
# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [6]:
# Checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

##### Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  

#### The event_datafile_new.csv contains the following columns: 
- artist; 
- firstName of user;
- gender of user;
- item number in session;
- last name of user;
- length of the song;
- level (paid or free song);
- location of the user;
- sessionId;
- song title;
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

Revisiting the dataset with pandas:

In [7]:
df = pd.read_csv('./event_datafile_new.csv')
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88


## Apache Cassandra

#### Creating a Cluster

In [8]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [9]:
# Creating a Keyspace 
session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
"""
)

#### Set Keyspace

In [10]:
# Setting KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

---
#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

1. Give me the **artist**, **song title** and **song's length** in the music app history that was heard during **sessionId = 338**, and **itemInSession  = 4**;


2. Give me only the following: **name of artist**, **song** (sorted by **itemInSession**) and **user (first and last name)** for **userid = 10**, **sessionid = 182**;
    

3. Give me every **user name (first and last)** in my music app history **who listened to the song 'All Hands Against His Own'**.




---
## Query 1

### Give me:
- artist; 
- song title; and 
- song's length... 

...in the music app history that was heard during `sessionId = 338`, and `itemInSession = 4`.

The **objetive** of this query is to retrieve information about a specific song listened by a user (logged at `sessionId`) and its position in this session listening queue. As a result you should see the `artist_name`, the `song_title` and the `song_length` (in seconds). The modeling of such query is build using as **(Composite) Primary Key** both the `sessionId` and `itemInSession` which are the information that would be inputted during the query.

Such query could be used as information to configure the recommendation system for the user, understanding how a specific song contributes to the listening time of a user in the application.

In [11]:
# Utils -- run as needed

# session.execute("DROP TABLE music_library")

##### Creating `music_library` table:

In [12]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Common CREATE TABLE statement
create_if_not_exists = "CREATE TABLE IF NOT EXISTS {}"

# Creating table for music_library
create_music_libary_table = create_if_not_exists.format(
                                                        """
                                                            music_library ( \
                                                                    sessionId INT, \
                                                                    itemInSession INT, \
                                                                    artist_name TEXT, \
                                                                    song_title TEXT, \
                                                                    song_length FLOAT, \
                                                                    PRIMARY KEY(sessionId, itemInSession) \
                                                                ); \
                                                        """
                                                       )



# Executing query
session.execute(create_music_libary_table)

# print(create_music_libary_table)

##### Populating `music_library` table:

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Building query code (as string) to be executed later
        query = "INSERT INTO music_library (sessionId, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        # Executing query saved above
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Doing a SELECT to verify that the data have been propperly inserted into the table

In [14]:
## Verifying SELECT statement to check if data was entered into the table correctly

select_query = """
    SELECT artist_name, song_title, song_length FROM music_library WHERE sessionId = 338 and itemInSession = 4; 
"""

rows = session.execute(select_query)

for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


---

### Query 2

#### Give me only the following: 

- name of artist, 
- song (sorted by itemInSession) and 
- user (first and last name) 

for `userid = 10`, `sessionid = 182`.

This query will be able to list the order which the user has listened to the songs. 

For that it will be constructed using using `userId` and `sessionId`, and this information will be used as **PRIMARY KEY** for the query. On the other hadn, the `itemInSession` information will also be included as part of the **Composite Partition Key** in the PK, acting as a [**CLUSTERING COLUMN**](https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useCompositePartitionKeyConcept.html#useCompositePartitionKeyConcept).


In [15]:
# Utils -- uncomment to run as needed

# session.execute("DROP TABLE artist_song_user")

# for i, column in enumerate(df.columns):
#     print(i, column)

##### Creating `artist_song_user` table:

In [16]:
# Query 2: 
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# Creating artist_song_user table
create_artist_song_user_table = create_if_not_exists.format(
                                                        """
                                                            artist_song_user ( \
                                                                    userId INT, \
                                                                    sessionId INT, \
                                                                    itemInSession INT, \
                                                                    artist TEXT, \
                                                                    firstName TEXT, \
                                                                    lastName TEXT, 
                                                                    song TEXT, \
                                                                    PRIMARY KEY((userId, sessionId), itemInSession) \
                                                                ); \
                                                        """
                                                       )



# Executing create_artist_song_user_table query
session.execute(create_artist_song_user_table)

##### Populating `artist_user_song` table:

In [17]:
# Populating table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Building query code
        query = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Executing query in the Cassandra session
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

##### Creating and running query:

In [18]:
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

select_artist_song_user_query = """
    SELECT artist, firstName, lastName, song, itemInSession \
    FROM artist_song_user \
    WHERE userId = 10 AND sessionId = 182;
"""

rows_artist_song_user_query = session.execute(select_artist_song_user_query)

for row in rows_artist_song_user_query:
    print(row)

Row(artist='Down To The Bone', firstname='Sylvie', lastname='Cruz', song="Keep On Keepin' On", iteminsession=0)
Row(artist='Three Drives', firstname='Sylvie', lastname='Cruz', song='Greece 2000', iteminsession=1)
Row(artist='Sebastien Tellier', firstname='Sylvie', lastname='Cruz', song='Kilometer', iteminsession=2)
Row(artist='Lonnie Gordon', firstname='Sylvie', lastname='Cruz', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3)


---

### Query 3

#### Give me only the following: 

- every **user name (first and last)** in my music app history who listened to the **song 'All Hands Against His Own'**

This query is build to list all users who've listened to a specific song. For that the **(Composite) Primary Key** is constructed combining both `song` and `userId` information. 

Since `userId`'s are already a unique identifier we will be able to select all users that listened to a particular song of interest.

In [20]:
# Utils -- run as needed

# session.execute("DROP TABLE user_song_AHAHO")

# for i, column in enumerate(df.columns):
#     print(i, column)

##### Creating `user_song_AHAHO` table:

In [21]:
## Query 3: 

# Creating user_song_AHAHO table
create_user_song_AHAHO = create_if_not_exists.format(
                                                        """
                                                            user_song_AHAHO ( \
                                                                    song TEXT, \
                                                                    userId INT, \
                                                                    firstName TEXT, \
                                                                    lastName TEXT, 
                                                                    PRIMARY KEY(song, userId) \
                                                                ); \
                                                        """
                                                       )



# Executing create_artist_song_user_table query
session.execute(create_user_song_AHAHO)

##### Populating `user_song_AHAHO` table:

In [22]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # Query to insert values into table
        query = "INSERT INTO user_song_AHAHO (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        # Executing query
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

##### Writing and running query:

In [23]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

select_user_song_AHAHO = """
    SELECT userId, firstName, lastName, song \
    FROM user_song_AHAHO \
    WHERE song = 'All Hands Against His Own';
"""

rows_artist_song_user_query = session.execute(select_user_song_AHAHO)

for row in rows_artist_song_user_query:
    print(row)

Row(userid=29, firstname='Jacqueline', lastname='Lynch', song='All Hands Against His Own')
Row(userid=80, firstname='Tegan', lastname='Levine', song='All Hands Against His Own')
Row(userid=95, firstname='Sara', lastname='Johnson', song='All Hands Against His Own')


### Drop the tables before closing out the sessions

In [24]:
## TO-DO: Drop the table before closing out the sessions

tables = ['music_library', 'artist_song_user', 'user_song_AHAHO']

for table in tables:
    session.execute("DROP TABLE {}".format(table))

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()